**SISTEMA DE RECOMENDACION - MODELO DE MACHINE LEARNING**

*En esta notebook se desarrolla el modelo de Machine Learning a llevar a cabo para recomendar peliculas*

En primer lugar, se va acotar el DATA SET unicamente a las columnas que vamos a utilizar en el modelo. 

Segun la investigacion realizada existen diferentes sistemas para recomendar peliculas:
- Basados en los usuarios: En este caso se guian por puntajes, popularidad y opiniones de los usuarios.
- Basados en el contentido: se guian en las similitudes que se pueden encontrar en el contenido de las caracteristicas que hacen a una pelicula, como por ejemplo: Actor, Director, Genero, Titulo, Palabras Clave, etc. 


En el caso del presente modelo, y segun el EDA, vemos que las columnas Populatiry y vote_count tienen muchos valores nulo y en muchos casos poca cantidad de votos, por lo que no se considera adecuado aplicar un sistema de recomendacion basado en usuarios. 

Por lo tanto, el sistema a desarrollar sera basado en el contenido.

En particular, para simplificar el Modelo y tomarlo como una herramienta de aprendizaje, se tendran en cuenta las siguientes caracteristicas:

Titulo, Genero, Actor, Director, Lenguaje Original

In [4]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
#importamos librerias
import pandas as pd
import os


In [6]:
#importamos el csv y lo transformamos en dataframe
ruta_archivo = os.path.join('Dataset', 'df_eda.csv')    #Lo descargamos con ruta relativa a la carpeta actual
df_ml = pd.read_csv(ruta_archivo)


In [7]:
# Crear una copia del DataFrame original para preservar los datos
df_copy = df_ml.copy()

# Seleccionar las columnas deseadas en df_copy
columns_to_keep = ['original_language', 'title', 'collection_name', 'genre_name', 'Director', 'first_actor']
df_copy = df_copy.loc[:, columns_to_keep]


In [8]:
df_copy.head(15)

,original_language,title,collection_name,genre_name,Director,first_actor
0,en,Waiting to Exhale,No hay datos,"['Comedy', 'Drama', 'Romance']",Forest Whitaker,Whitney Houston
1,en,Sabrina,No hay datos,"['Comedy', 'Romance']",Sydney Pollack,Harrison Ford
2,en,Tom and Huck,No hay datos,"['Action', 'Adventure', 'Drama', 'Family']",David Loughery,Jonathan Taylor Thomas
3,en,Sudden Death,No hay datos,"['Action', 'Adventure', 'Thriller']",Peter Hyams,Jean-Claude Van Damme
4,en,The American President,No hay datos,"['Comedy', 'Drama', 'Romance']",John Seale,Michael Douglas
5,en,Dracula: Dead and Loving It,No hay datos,"['Comedy', 'Horror']",Adam Weiss,Leslie Nielsen
6,en,Nixon,No hay datos,"['History', 'Drama']",Oliver Stone,Anthony Hopkins
7,en,Cutthroat Island,No hay datos,"['Action', 'Adventure']",Peter Levy,Geena Davis
8,en,Ace Ventura: When Nature Calls,Ace Ventura Collection,"['Crime', 'Comedy', 'Adventure']",Steve Oedekerk,Jim Carrey
9,en,Money Train,No hay datos,"['Action', 'Comedy', 'Crime']",Joseph Ruben,Wesley Snipes


In [9]:
# Cambiar el orden de las columnas en df_copy. Esto lo hacemos por una cuestion de orden nada mas y de coherencia
column_order = ['title', 'genre_name', 'original_language', 'first_actor', 'Director']
df_copy = df_copy.reindex(columns=column_order)


In [10]:
df_copy.sample(50)

,title,genre_name,original_language,first_actor,Director
15047,The Scapegoat,[],en,Andrew Scott,Charles Sturridge
3689,Happy Times,"['Drama', 'Comedy']",zh,Zhao Benshan,Zhang Yimou
21224,3:15,"['Action', 'Crime', 'Thriller']",en,Adam Baldwin,Larry Gross
5636,Freedom Downtime,"['Drama', 'Documentary', 'Foreign']",en,Sean Gullette,Emmanuel Goldstein
14447,"RKO Production 601: The Making of 'Kong, the E...",[],en,Sam Fontana,Phil Tippett
20288,Suing The Devil,"['Comedy', 'Drama', 'Thriller']",en,Malcolm McDowell,Timothy A. Chey
15095,Zatôichi Meets Yojimbo,"['Adventure', 'Action', 'Drama']",ja,Shintarô Katsu,Kan Shimosawa
4237,A Patch of Blue,"['Drama', 'Romance']",en,Sidney Poitier,Guy Green
5649,Distant Drums,"['Action', 'Western']",en,Gary Cooper,Max Steiner
7801,Casper Meets Wendy,"['Family', 'Comedy', 'Adventure']",en,Cathy Moriarty,Rob Kerchner


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import ast

def content_based_recommender(movie_title, df):
    # Crear un objeto TfidfVectorizer para convertir el texto en vectores
    vectorizer = TfidfVectorizer(
        tokenizer=lambda x: x.split(' '), # Personalizamos el tokenizer para que no separe las palabras del género
        stop_words='english', # Agregamos stop words para eliminar palabras comunes que no aportan mucho significado
        use_idf=True, # Utilizamos la frecuencia inversa del documento (IDF) para dar más peso a las palabras poco frecuentes
        smooth_idf=True, # Evitamos dividir por cero al suavizar el IDF
        norm=None, # No normalizamos los vectores para poder ajustar manualmente la relevancia de 'genre_name'
        sublinear_tf=True, # Aplicamos sublineal TF para suavizar la frecuencia de términos en el texto
        min_df=0.001 # Filtro de términos que aparecen en menos del 0.1% de los documentos
    )

    # Obtener las columnas relevantes del DataFrame
    df_relevant = df[['title', 'genre_name', 'Director', 'first_actor', 'original_language']]

    # Preprocesar las columnas de texto para su uso en el modelo
    df_relevant['genre_name'] = df_relevant['genre_name'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
    df_relevant['text'] = df_relevant['title'] + ' ' + df_relevant['genre_name'].apply(lambda x: ' '.join(x)) + ' ' + df_relevant['Director'] + ' ' + df_relevant['first_actor'] + ' ' + df_relevant['original_language']

    # Calcular la matriz de vectores TF-IDF
    tfidf_matrix = vectorizer.fit_transform(df_relevant['text'])

    # Calcular la similitud coseno entre las películas
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Obtener el índice de la película de entrada si existe
    if movie_title in df_relevant['title'].values:
        idx = df_relevant[df_relevant['title'] == movie_title].index[0]
    else:
        print("La película no se encuentra en el conjunto de datos.")
        return None

    # Obtener las puntuaciones de similitud coseno para todas las películas
    scores = list(enumerate(cosine_similarities[idx]))

    # Ordenar las películas por puntuación de similitud
    scores = sorted(scores, key=lambda x: (x[1], x[0]), reverse=True)

    # Obtener las 5 películas recomendadas
    top_recommendations = scores[1:6]  # Excluir la película de entrada

    # Crear un diccionario con los datos de las películas recomendadas
    recommendations = []
    for idx, score in top_recommendations:
        title = df_relevant.iloc[idx]['title']
        genre = df_relevant.iloc[idx]['genre_name']
        director = df_relevant.iloc[idx]['Director']
        actor = df_relevant.iloc[idx]['first_actor']
        language = df_relevant.iloc[idx]['original_language']
        recommendation = {
            'Title': title,
            'Genre': genre,
            'Director': director,
            'Actor': actor,
            'Language': language
        }
        recommendations.append(recommendation)

    # Obtener los datos de la película de entrada
    movie_data = df_relevant[df_relevant['title'] == movie_title].iloc[0]
    movie_info = {
        'Title': movie_data['title'],
        'Genre': movie_data['genre_name'],
        'Director': movie_data['Director'],
        'Actor': movie_data['first_actor'],
        'Language': movie_data['original_language']
    }

    # Imprimir los datos de la película de entrada
    print("Película:", movie_title)
    print("Título:", movie_info['Title'])
    print("Género:", movie_info['Genre'])
    print("Director:", movie_info['Director'])
    print("Actor:", movie_info['Actor'])
    print("Idioma:", movie_info['Language'])

    # Imprimir las películas recomendadas
    print("\nRecomendaciones para la película:", movie_title)
    for i, recommendation in enumerate(recommendations):
        print(f"{i+1}. {recommendation['Title']} - Género: {recommendation['Genre']}, Director: {recommendation['Director']}, Actor: {recommendation['Actor']}, Idioma: {recommendation['Language']}")

    return recommendations



In [24]:

# Ejemplo de uso:
movie_title = 'Sabrina'
recommendations = content_based_recommender(movie_title, df_copy)

/Users/compu/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Película: Sabrina
Título: Sabrina
Género: ['Comedy', 'Romance']
Director: Sydney Pollack
Actor: Harrison Ford
Idioma: en

Recomendaciones para la película: Sabrina
1. Sabrina - Género: ['Comedy', 'Romance'], Director: Sydney Pollack, Actor: Harrison Ford, Idioma: en
2. Random Hearts - Género: ['Drama', 'Romance'], Director: Philippe Rousselot, Actor: Harrison Ford, Idioma: en
3. The Star Wars Holiday Special - Género: ['Adventure', 'Comedy', 'Family', 'Science Fiction', 'TV Movie'], Director: Steve Binder, Actor: Harrison Ford, Idioma: en
4. Living in the Age of Airplanes - Género: ['Documentary'], Director: Brian J. Terwilliger, Actor: Harrison Ford, Idioma: en
5. Extraordinary Measures - Género: ['Drama'], Director: Robert Nelson Jacobs, Actor: Harrison Ford, Idioma: en


**RESUMEN Y EXPLICACION DEL MODELO UTILIZADO**

El modelo proporcionado es un sistema de recomendación basado en contenido. Utiliza técnicas de procesamiento de lenguaje natural (NLP) y similitud coseno para encontrar películas similares en función de características de texto como el título, el género, el director, el actor principal y el idioma original.

A continuación, se explica cómo funciona el modelo paso a paso:

1. Se crea un objeto `TfidfVectorizer` de la biblioteca `sklearn` para convertir el texto en vectores. Se configuran varios parámetros, como el tokenizador, las palabras comunes a eliminar, el uso del IDF, la suavización del IDF y el filtro de términos infrecuentes.

2. Se seleccionan las columnas relevantes del DataFrame de películas, que son el título, el género, el director, el actor principal y el idioma original.

3. Se realiza un preprocesamiento en las columnas de texto para su uso en el modelo. Se convierten las listas de géneros de texto en listas reales utilizando la función `ast.literal_eval`. Luego, se crea una columna adicional llamada "texto" que combina el título, el género, el director, el actor principal y el idioma original en una sola cadena de texto.

4. Se calcula la matriz de vectores TF-IDF utilizando el objeto `TfidfVectorizer` y el texto preprocesado. Cada película se representa como un vector en el espacio vectorial TF-IDF, donde las palabras raras tienen un peso mayor.

5. Se calcula la similitud coseno entre todas las películas utilizando la matriz de vectores TF-IDF. El resultado es una matriz de similitud que indica qué tan similares son las películas entre sí en función de su contenido de texto.

6. Se busca el índice de la película de entrada en el DataFrame. Si el título de la película está en el conjunto de datos, se obtiene su índice correspondiente.

7. Se calculan los puntajes de similitud coseno entre la película de entrada y todas las demás películas. Los puntajes representan qué tan similares son las películas en comparación con la película de entrada.

8. Se ordenan las películas según los puntajes de similitud coseno, de mayor a menor.

9. Se seleccionan las 5 películas con los puntajes más altos como recomendaciones principales, excluyendo la película de entrada.

10. Se crea una lista de diccionarios con los datos de las películas recomendadas, incluyendo el título, el género, el director, el actor principal y el idioma original.

11. Se muestra la información de la película de entrada, como el título, el género, el director, el actor principal y el idioma original.

12. Se muestran las películas recomendadas junto con su información relevante, como el título, el género, el director, el actor principal y el idioma original.

En resumen, el modelo utiliza la similitud coseno en vectores TF-IDF para encontrar películas similares en función del contenido de texto, como el título, el género, el director y el actor principal. Las películas con los puntajes de similitud más altos se seleccionan como recomendaciones principales.